In [ ]:
from typing import Optional

def keep(value: Optional[str]) -> Optional[str]:
    return value

def nullify(value: Optional[str]) -> None:
    return None

def mask_fixed(value: Optional[str], mask_char: str = "X") -> Optional[str]:
    if value is None:
        return None
    return mask_char * len(str(value))

def mask_lastn(value: Optional[str], n: int = 4, mask_char: str = "X") -> Optional[str]:
    if value is None:
        return None
    s = str(value)
    if len(s) <= n:
        return s
    return (mask_char * (len(s) - n)) + s[-n:]

def mask_firstn(value: Optional[str], n: int = 4, mask_char: str = "X") -> Optional[str]:
    if value is None:
        return None
    s = str(value)
    if len(s) <= n:
        return s
    return s[:n] + (mask_char * (len(s) - n))

In [ ]:
from snowflake.snowpark import Session
session = Session.builder.getOrCreate()

# Session is already created in Notebook as `session`
cfg_df = session.table("PUBLIC_OBF.OBF_CFG_COLUMNS") \
    .filter("dataset_name = 'TPCH_CUSTOMER_5RULES' AND enabled = TRUE")

cfg = cfg_df.collect()

In [ ]:
import json

def apply_rule_value(rule_name: str, value, obf_params):
    """Dispatch obfuscation rule to the right Python function."""
    params = {}
    if obf_params is not None:
        params = json.loads(str(obf_params))  # VARIANT → dict

    if rule_name == "KEEP":
        return keep(value)
    elif rule_name == "NULLIFY":
        return nullify(value)
    elif rule_name == "MASK_FIXED":
        return mask_fixed(value, **params)
    elif rule_name == "MASK_LASTN":
        return mask_lastn(value, **params)
    elif rule_name == "MASK_FIRSTN":
        return mask_firstn(value, **params)
    else:
        return value  # fallback (no change)

In [ ]:
# Load 10 sample rows from CUSTOMER
cust_df = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER").limit(10)
cust_pd = cust_df.to_pandas()

# Apply transformations column by column based on config
cust_obf = cust_pd.copy()
for row in cfg:
    col = row["COLUMN_NAME"]
    rule = row["OBF_RULE"]
    params = row["OBF_PARAMS"]
    cust_obf[col] = cust_obf[col].apply(lambda v: apply_rule_value(rule, v, params))

cust_obf